In [10]:
from importance_measures.bdds import Buddy, BuddyNode, influence, blame, omega
import time
import random

def random_k_sat(n,m,k):
    # n variables, m clauses, k literals per clause
    clauses = []
    for _ in range(m):
        ret = []
        for _ in range(k):
            v = random.randrange(n)
            polarity = random.randint(0, 1)
            ret.append((polarity, v)) 
        clauses.append(ret)
    vars = [ f"x{idx}" for idx in range(n)]
    return vars, "&".join(
        "(" + "|".join(
            vars[idx] if pol==1 else "~" + vars[idx] for 
            (pol, idx) in cl
        ) + ")" for cl in clauses) 

def time_op(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        val = func(*args, **kwargs)
        dt_s = (time.time() - start)
        print(f"[{dt_s*1000:010.4f} ms / {dt_s:04.4f} s / {dt_s/60:02.4f} min] {func.__name__}") # {args}")
        return val
    return wrapper

vars, formula = random_k_sat(15,15,4)
# print("formula")
# print("="*10)
# print("&\n".join(formula.split("&")))
# print("="*10)
with Buddy(vars) as model:
    x = vars[0]
    f = model.node(formula)
    # f = model.node("x0 | (x1 & x2)")
    print(f"bdd size = {f.nodecount}")
    f.dump("f.pdf")
    blame, ub = time_op(blame)(f, x, rho=lambda x: 2**-x, cutoff=0.01, debug=True)
    infl = time_op(influence)(f, x)
    g = time_op(omega)(f)
    # g.dump("omega.pdf")
    print(f"influence {infl:0.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")

bdd size = 223
=== COMPUTING BLAME for x0 in BDD with size 223 ===
k 0 size g 173 size ell 173 d result 0.3087 max increase possible 0.3456
k 1 size g 449 size ell 509 d result 0.1293 max increase possible 0.1081
k 2 size g 335 size ell 383 d result 0.0211 max increase possible 0.0435
k 3 size g 252 size ell 95 d result 0.0003 max increase possible 0.0216
=== DONE ===
[00702.3072 ms / 0.7023 s / 0.0117 min] blame
[00000.1357 ms / 0.0001 s / 0.0000 min] influence
[00015.1331 ms / 0.0151 s / 0.0003 min] omega
influence 0.30872, 0.45950 <= blame <= 0.45950, blame diff 0.00000
